<a href="https://colab.research.google.com/github/ykim71/google_toxicity/blob/main/google_toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load File (can be done locally)

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd drive/'MyDrive'/toxicity/

/content/drive/MyDrive/toxicity


## load file from local

In [ ]:
from google.colab import files

uploaded = files.upload()


# Perpective API toxicity 



> Language Attributes: https://developers.perspectiveapi.com/s/about-the-api-attributes-and-languages



> API Request: https://developers.perspectiveapi.com/s/docs-get-started (note UT Google Account may not work; recommend using personal Google account for request)



In [4]:

from googleapiclient import discovery
from googleapiclient.errors import HttpError


In [5]:
API_KEY='YOUR-API-KEY'


In [6]:

# variable descriptions: https://github.com/conversationai/perspectiveapi
# you can replace toxicity attributes here:
analyze_request = {
   'comment': { 'text': 'xx'}, # setting formats (id, text)
   'requestedAttributes': {'TOXICITY@6': {}, # see the actual variable name from the Perspective API page
                           'LIKELY_TO_REJECT@2': {}, 
                           'INSULT': {}, 
                           'IDENTITY_ATTACK': {} 
                           },
   'doNotStore': True, # for other settings, https://developers.perspectiveapi.com/s/about-the-api-methods
}



In [7]:
import pandas as pd

sample_text = pd.read_csv('sample_code_review.csv')


In [8]:
# take random 3 samples to see if data has loaded successfully; 'text' is the column that you want to analyze

sample_text.sample(3)

,uid_sample,text
14,1483,Feel good piece. Father And Son Graduate Moreh...
0,921,A little bit of American truth? Mike Pence: Th...
18,1498,"Left Action Fight back, and tell Boehner & M..."


In [11]:
# setting global variables; setting the column name from the dataset; 

TEXT = 'text' 

In [13]:
import csv
import codecs
import json
import time
import pandas as pd

# setting attributes, can add more attiributes 

service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)

start = time.time()

comments_toxicity_list = []
comments_reject_list = []
comments_insult_list = []
comments_identity_list = []

for i in sample_text[TEXT].values.tolist(): 
  analyze_request['comment']['text'] = i
  
  try:
    response = service.comments().analyze(body=analyze_request).execute()
    i = json.loads(json.dumps(response, indent=2))
    
    comments_toxicity = i['attributeScores']['TOXICITY@6']['summaryScore']['value']
    comments_reject = i['attributeScores']['LIKELY_TO_REJECT@2']['summaryScore']['value']
    comments_insult = i['attributeScores']['INSULT']['summaryScore']['value']
    comments_identity = i['attributeScores']['IDENTITY_ATTACK']['summaryScore']['value']
        
  except HttpError:
    comments_toxicity = "error"
    comments_reject = "error"
    comments_insult = "error"
    comments_identity = "error"
            
  comments_toxicity_list.append(comments_toxicity)
  comments_reject_list.append(comments_reject)
  comments_insult_list.append(comments_insult)
  comments_identity_list.append(comments_identity)
        
sample_text = sample_text.join(pd.DataFrame({'toxicity': comments_toxicity_list, 
                                             'reject': comments_reject_list, 
                                             'insult': comments_insult_list, 
                                             'attack': comments_identity_list}))

end=time.time()
print("complete time: ", round(end -start, 2))

complete time:  0.88


In [14]:
sample_text.sample(3)

,uid_sample,text,toxicity,reject,insult,attack
0,921,A little bit of American truth? Mike Pence: Th...,0.277589,0.836571,0.118428,0.213282
16,1486,"During an interview with CNBC on Wednesday, Tr...",0.041074,0.331779,0.126426,0.021339
20,1819,Gonna leave this one right here AFROPUNK üôà...,0.153451,0.973788,0.262546,0.448278


In [ ]:
sample_text.to_csv('toxicity.csv')